# MNIST handwritten digits classification with CNNs

In this notebook, we'll train a convolutional neural network (CNN, ConvNet) to classify MNIST digits using Keras.

First, the needed imports. Note that there are a few new layers compared to the MLP notebook: Flatten, Convolution2D, MaxPooling2D.

In [ ]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# one-hot encoding:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print()
print('MNIST data loaded: train:',len(X_train),'test:',len(X_test))
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('Y_train:', Y_train.shape)

We'll have to do a bit of tensor manipulations, depending on the used backend (Theano or Tensorflow).

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print('X_train:', X_train.shape)

Now we are ready to create a convolutional model.

The `Convolution2D` layers operate on 2D matrices so we input the digit images directly to the model.  

The `MaxPooling2D` layer reduces the spatial dimensions, that is, makes the image smaller.

The `Flatten` layer flattens the 2D matrices into vectors, so we can then switch to  `Dense` layers as in the MLP model. 

See https://keras.io/layers/convolutional/, https://keras.io/layers/pooling/ for more information.

In [ ]:
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

Now let's train the CNN model. Note that we do not need the `reshape()` function as in the MLP case. 

This is a relatively complex model, so training is considerably slower than with MLPs. 

In [ ]:
nb_epoch = 3 # one epoch takes about 80 seconds

history = model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=128)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['acc'])
plt.title('accuracy')

With enough training epochs, the test accuracy should exceed 99%.  

You can compare your result with the state-of-the art [here](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html).  Even more results can be found [here](http://yann.lecun.com/exdb/mnist/). 

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=1)
print()
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
predictions = model.predict(X_test)
rounded = np.argmax(predictions, axis=1)
errors = rounded != y_test
nerrors = np.count_nonzero(errors)
print('Wrong predictions:', nerrors)

maxtoshow = 10
print('Showing', maxtoshow, 'first:')
ii = 0
for i in range(X_test.shape[0]):
    if ii>=maxtoshow:
        break
    if errors[i]:
        plt.figure(figsize=(1, 1))
        plt.axis('off')
        if K.image_dim_ordering() == 'th':
            plt.imshow(X_test[i,0,:,:], cmap="gray")
        else:
            plt.imshow(X_test[i,:,:,0], cmap="gray")
        print(ii,': correct:',y_test[i],'predicted:', rounded[i])
        ii = ii + 1

## Bonus: train the model in taito-gpu

The above model can also be run in taito-gpu in a couple of easy steps:

```sh
ssh -l USERNAME taito-gpu.csc.fi

module purge 
module load python-env/2.7.10 cuda/8.0
    
# the following two commands need to be entered only once
PYTHONUSERBASE=$USERAPPL/tensorflow.0.11.0 pip install --user /wrk/jppirhon/tensorflow.0.11.0-gcc493_pkg/tensorflow-0.11.0-py2-none-any.whl
pip install --user keras h5py Pillow
    
export PYTHONPATH=$USERAPPL/tensorflow.0.11.0/lib/python2.7/site-packages
sbatch /wrk/makoskel/run-python27-gputest.sh /wrk/makoskel/keras-mnist-cnn-taitogpu.py
```

One epoch should take about 8 seconds.  With 10 epochs, the model should have > 99% accuracy.